In [12]:
import pandas as pd
import os
from datetime import datetime, timedelta
import shutil
import time
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import warnings
from bs4 import BeautifulSoup



from bs4 import BeautifulSoup

# MSP BLOG

# open Chrome -> naver -> login

In [9]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

try:
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#id')))
    time.sleep(3)
    nid = 'qli_2018'  # Your ID
    npw = '12341234!@'  # Your Password
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR, '#id').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    pyperclip.copy(npw)
    browser.find_element(By.CSS_SELECTOR, '#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy('')  # Clear clipboard for security
    browser.find_element(By.XPATH, '//*[@id="log.login"]').click()
    browser.get('https://admin.blog.naver.com/mysecondplace/stat/today')  # Navigate to the target page

    # Add your code here to parse the page with BeautifulSoup or perform other actions

except Exception as e:
    print(f"Exception: {str(e)}")

# article_ids listup

In [3]:
# 'ContentsList.xlsx' 파일 불러오기
file_path = r'C:\Users\ch.kang\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\ContentsList.xlsx'
df = pd.read_excel(file_path)
df.columns

Index(['20240119', 'contents_url', 'category', 'board1', 'board2', 'title',
       'post_start', 'post_end'],
      dtype='object')

In [4]:
# 첫 번째 열이 'BH'로 시작하는 행을 선택
selected_rows = df[df[df.columns[0]].str.startswith('BM')]

# contents_url 열에서 뒤에 12 값을 리스트로 추출
article_ids = selected_rows['contents_url'].str[-12:].tolist()

# 중복된 값을 제거하고 정렬
article_ids = list(set(article_ids))
article_ids.sort()

In [10]:
article_ids = ['223203815685',
 '223208249149',
 '223209462852']

# crawler for "views", "Acquisition"

In [19]:
# Initialize view, total view, total like, and total comment variables
view_data = []
total_views = []
total_likes = []
total_comments = []

# Create a Selenium WebDriver instance
for article_id in article_ids:
    # Navigate to the article page
    url = f"https://blog.stat.naver.com/blog/article/{article_id}/referer"
    browser.get(url)

    # Click necessary buttons
    # 어제 날짜 클릭
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_info_section__1U7kD > div > div:nth-child(1) > div > a").click()

    # Wait until the view counts, like counts, and comment counts are loaded
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(1) > strong"))
    )

    # Extract total views, total likes, and total comments using browser.find_element
    total_views_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(1) > strong")
    total_likes_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(1) > strong")
    total_comments_element = browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_data_section__3AZ-7 > div > div:nth-child(2) > ul > li:nth-child(2) > strong")

    # Get the text values from the elements
    total_views_text = total_views_element.text.strip()
    total_likes_text = total_likes_element.text.strip()
    total_comments_text = total_comments_element.text.strip()

    # Append the data to the respective lists
    total_views.append({'article_id': article_id, 'total_views': total_views_text})
    total_likes.append({'article_id': article_id, 'total_likes': total_likes_text})
    total_comments.append({'article_id': article_id, 'total_comments': total_comments_text})
 
    #Wait until the "조회수" element is present
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong"))
    )
    time.sleep(1)
    # Extract the text from the element
    extracted_text = element.text.strip()
    view_data.append({'article_id': article_id, 'views': extracted_text})

    # 다운로드 페이지 클릭
    time.sleep(3)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)").click()

    #"유입분석" 클릭
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label").click()
    
    #"지표 다운로드" 클릭
    time.sleep(2)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_btn_section__3dOcL > a.u_ni_btn_download__KmUvf.u_ni_type_green").click()
    
    #Rename and move the downloaded file
    time.sleep(5)
    latest_file = max([f for f in os.listdir(download_directory) if f.endswith('.xlsx')], key=lambda x: os.path.getctime(os.path.join(download_directory, x)))
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
    new_file_name = f'{article_id}_{yesterday}.xlsx'
    os.rename(os.path.join(download_directory, latest_file), os.path.join(download_directory, new_file_name))
    desired_folder = r'C:\Users\ch.kang\Documents\DB_OriginalData_xlsx\NaverBlogStatistics_MSP'
    shutil.move(os.path.join(download_directory, new_file_name), os.path.join(desired_folder, new_file_name))


In [23]:
view_data

[{'article_id': '223203815685', 'views': '3'},
 {'article_id': '223208249149', 'views': '7'},
 {'article_id': '223209462852', 'views': '3'}]

In [20]:
total_views

[{'article_id': '223203815685', 'total_views': '67'},
 {'article_id': '223208249149', 'total_views': '366'},
 {'article_id': '223209462852', 'total_views': '113'}]

In [21]:
total_likes

[{'article_id': '223203815685', 'total_likes': '6'},
 {'article_id': '223208249149', 'total_likes': '5'},
 {'article_id': '223209462852', 'total_likes': '3'}]

In [22]:
total_comments

[{'article_id': '223203815685', 'total_comments': '0'},
 {'article_id': '223208249149', 'total_comments': '0'},
 {'article_id': '223209462852', 'total_comments': '0'}]

In [5]:
view_data = []
download_directory = r'C:\Users\ch.kang\Downloads'

# Create a Selenium WebDriver instance
for article_id in article_ids:
    # Navigate to the article page
    url = f"https://blog.stat.naver.com/blog/article/{article_id}/referer"
    browser.get(url)

    # Click necessary buttons
    # 어제 날짜 클릭
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR,"#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_info_section__1U7kD > div > div:nth-child(1) > div > a").click()
 
    #Wait until the "조회수" element is present
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong"))
    )
    time.sleep(1)
    # Extract the text from the element
    extracted_text = element.text.strip()
    view_data.append({'article_id': article_id, 'views': extracted_text})

    # 다운로드 페이지 클릭
    time.sleep(3)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)").click()

    #"유입분석" 클릭
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label").click()
    
    #"지표 다운로드" 클릭
    time.sleep(2)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_btn_section__3dOcL > a.u_ni_btn_download__KmUvf.u_ni_type_green").click()
    
    #Rename and move the downloaded file
    time.sleep(5)
    latest_file = max([f for f in os.listdir(download_directory) if f.endswith('.xlsx')], key=lambda x: os.path.getctime(os.path.join(download_directory, x)))
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
    new_file_name = f'{article_id}_{yesterday}.xlsx'
    os.rename(os.path.join(download_directory, latest_file), os.path.join(download_directory, new_file_name))
    desired_folder = r'C:\Users\ch.kang\Documents\DB_OriginalData_xlsx\NaverBlogStatistics_MSP'
    shutil.move(os.path.join(download_directory, new_file_name), os.path.join(desired_folder, new_file_name))
browser.close()    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF6FC1E2142+3514994]
	(No symbol) [0x00007FF6FBE00CE2]
	(No symbol) [0x00007FF6FBCA76AA]
	(No symbol) [0x00007FF6FBC80AFD]
	(No symbol) [0x00007FF6FBD1CB1B]
	(No symbol) [0x00007FF6FBD3218F]
	(No symbol) [0x00007FF6FBD15D93]
	(No symbol) [0x00007FF6FBCE4BDC]
	(No symbol) [0x00007FF6FBCE5C64]
	GetHandleVerifier [0x00007FF6FC20E16B+3695259]
	GetHandleVerifier [0x00007FF6FC266737+4057191]
	GetHandleVerifier [0x00007FF6FC25E4E3+4023827]
	GetHandleVerifier [0x00007FF6FBF304F9+689705]
	(No symbol) [0x00007FF6FBE0C048]
	(No symbol) [0x00007FF6FBE08044]
	(No symbol) [0x00007FF6FBE081C9]
	(No symbol) [0x00007FF6FBDF88C4]
	BaseThreadInitThunk [0x00007FFDBB3F257D+29]
	RtlUserThreadStart [0x00007FFDBC06AA58+40]


# Acquisition data preprocessing

In [ ]:
# Suppress openpyxl warning about missing default style
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.styles.stylesheet")

# Directory containing the Excel files
directory_path = r'C:\Users\ch.kang\Documents\DB_OriginalData_xlsx\NaverBlogStatistics_MSP'

# List of Excel files in the directory
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx') and yesterday in file]

# 데이터를 저장할 딕셔너리
data_dict = {}

# 각 엑셀 파일을 읽어서 데이터를 딕셔너리에 저장
for excel_file in excel_files:
    # 엑셀 파일의 경로
    excel_file_path = os.path.join(directory_path, excel_file)
    
    # 엑셀 파일 이름에서 14~21자리를 날짜 정보로 추출
    date_info = excel_file[13:22]

    # Extract article_id from the Excel file name
    article_id = excel_file[:12]
    
    # 엑셀 파일을 데이터프레임으로 읽어오기
    df1 = pd.read_excel(excel_file_path)
    title = df1.iloc[4, 1]
    
    # 엑셀 파일을 데이터프레임으로 읽어오기
    df = pd.read_excel(excel_file_path, header=8)
    
    # 데이터프레임을 딕셔너리에 저장, 날짜 정보와 views 정보도 함께 저장
    data_dict[article_id] = {
    'title': title,
    'date_info': date_info,
    'views': None,
    'acq_path': df['유입경로'].tolist(),
    'acq_path_ratio': df['비율'].tolist(),
    'acq_path_detail': df['상세유입경로'].tolist(),
    'acq_path_detail_ratio': df['비율.1'].tolist()
    }

# views 정보를 view_data에서 가져와서 딕셔너리에 추가
for item in view_data:
    article_id = item['article_id']
    views = item['views']
    key = article_id[:12]  # article_id의 앞 12자리를 key로 사용
    
    if key in data_dict:
        # views 값을 숫자로 변환 (가능한 경우에만)
        try:
            views_count = int(views)
        except (ValueError, TypeError):
            views_count = 0  # 변환할 수 없는 경우 views_count를 0으로 설정

        data_dict[key]['views'] = views

        # acq_path 및 acq_path_detail 결합 (해당 필드가 존재하는 경우에만)
        if 'acq_path_ratio' in data_dict[key] and 'acq_path' in data_dict[key]:
            new_acq_path = [f"{path}({ratio}%, {views_count * ratio / 100}건)" for path, ratio in zip(data_dict[key]['acq_path'], data_dict[key]['acq_path_ratio'])]
            data_dict[key]['acq_path'] = new_acq_path

        if 'acq_path_detail_ratio' in data_dict[key] and 'acq_path_detail' in data_dict[key]:
            new_acq_path_detail = [f"{path}({ratio}%, {views_count * ratio / 100}건)" for path, ratio in zip(data_dict[key]['acq_path_detail'], data_dict[key]['acq_path_detail_ratio'])]
            data_dict[key]['acq_path_detail'] = new_acq_path_detail

# 'acq_path_detail_ratio' 및 'acq_path_ratio' 필드를 모든 data_dict 항목에서 제거
for key in data_dict:
    if 'acq_path_detail_ratio' in data_dict[key]:
        del data_dict[key]['acq_path_detail_ratio']
    if 'acq_path_ratio' in data_dict[key]:
        del data_dict[key]['acq_path_ratio']

# Output Preprocessed Data

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data_dict)

# 전치 수행
df_msp = df.T

In [ ]:
df_msp

In [ ]:
df_msp['category'] = 'blog_msp'
cols = ['category'] + [col for col in df_msp.columns if col != 'category']
df_msp = df_msp[cols]
df_msp

# HANPRO BLOG

# open Chrome -> naver -> login

In [ ]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

try:
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#id')))
    time.sleep(3)
    nid = 'hanpro911'  # Your ID
    npw = 'Zmfflwntlr1!'  # Your Password
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR, '#id').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    pyperclip.copy(npw)
    browser.find_element(By.CSS_SELECTOR, '#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy('')  # Clear clipboard for security
    browser.find_element(By.XPATH, '//*[@id="log.login"]').click()
    browser.get('https://admin.blog.naver.com/hanpro911/stat/today')  # Navigate to the target page

    # Add your code here to parse the page with BeautifulSoup or perform other actions

except Exception as e:
    print(f"Exception: {str(e)}")

# article_ids listup

In [ ]:
# 'ContentsList.xlsx' 파일 불러오기
file_path = r'C:\Users\ch.kang\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\ContentsList.xlsx'
df = pd.read_excel(file_path)
df.columns

In [ ]:
# 첫 번째 열이 'BH'로 시작하는 행을 선택
selected_rows = df[df[df.columns[0]].str.startswith('BH')]

# contents_url 열에서 뒤에 12 값을 리스트로 추출
article_ids = selected_rows['contents_url'].str[-12:].tolist()

# 중복된 값을 제거하고 정렬
article_ids = list(set(article_ids))
article_ids.sort()

# crawler for "views", "Acquisition"

In [ ]:
view_data = []
download_directory = r'C:\Users\ch.kang\Downloads'

# Create a Selenium WebDriver instance
for article_id in article_ids:
    # Navigate to the article page
    url = f"https://blog.stat.naver.com/blog/article/{article_id}/referer"
    browser.get(url)

    # Click necessary buttons
    # 어제 날짜 클릭
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR,"#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_info_section__1U7kD > div > div:nth-child(1) > div > a").click()
 
    #Wait until the "조회수" element is present
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_root > div > div > div:nth-child(2) > div.u_ni_stats_info_wrap > div > div.u_ni_data_section > div > ul > li:nth-child(1) > strong"))
    )
    time.sleep(1)
    # Extract the text from the element
    extracted_text = element.text.strip()
    view_data.append({'article_id': article_id, 'views': extracted_text})

    # 다운로드 페이지 클릭
    time.sleep(3)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div.u_ni_header_wrap > div > div.u_ni_index_section__2a-c1 > a > span:nth-child(2)").click()

    #"유입분석" 클릭
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_condition_section__1k56B > div > div > div.u_ni_option_group__JMFLr > form > ul:nth-child(2) > li:nth-child(1) > label").click()
    
    #"지표 다운로드" 클릭
    time.sleep(2)
    browser.find_element(By.CSS_SELECTOR, "#_root > div > div > div > div.u_ni_btn_section__3dOcL > a.u_ni_btn_download__KmUvf.u_ni_type_green").click()
    
    #Rename and move the downloaded file
    time.sleep(5)
    latest_file = max([f for f in os.listdir(download_directory) if f.endswith('.xlsx')], key=lambda x: os.path.getctime(os.path.join(download_directory, x)))
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
    new_file_name = f'{article_id}_{yesterday}.xlsx'
    os.rename(os.path.join(download_directory, latest_file), os.path.join(download_directory, new_file_name))
    desired_folder = r'C:\Users\ch.kang\Documents\DB_OriginalData_xlsx\NaverBlogStatistics_hanpro'
    shutil.move(os.path.join(download_directory, new_file_name), os.path.join(desired_folder, new_file_name))
browser.close()   

# Acquisition data preprocessing

In [ ]:
# Suppress openpyxl warning about missing default style
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.styles.stylesheet")

# Directory containing the Excel files
directory_path = r'C:\Users\ch.kang\Documents\DB_OriginalData_xlsx\NaverBlogStatistics_hanpro'

# List of Excel files in the directory
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx') and yesterday in file]

# 데이터를 저장할 딕셔너리
data_dict = {}

# 각 엑셀 파일을 읽어서 데이터를 딕셔너리에 저장
for excel_file in excel_files:
    # 엑셀 파일의 경로
    excel_file_path = os.path.join(directory_path, excel_file)
    
    # 엑셀 파일 이름에서 14~21자리를 날짜 정보로 추출
    date_info = excel_file[13:22]

    # Extract article_id from the Excel file name
    article_id = excel_file[:12]
    
    # 엑셀 파일을 데이터프레임으로 읽어오기
    df1 = pd.read_excel(excel_file_path)
    title = df1.iloc[4, 1]
    
    # 엑셀 파일을 데이터프레임으로 읽어오기
    df = pd.read_excel(excel_file_path, header=8)
    
    # 데이터프레임을 딕셔너리에 저장, 날짜 정보와 views 정보도 함께 저장
    data_dict[article_id] = {
    'title': title,
    'date_info': date_info,
    'views': None,
    'acq_path': df['유입경로'].tolist(),
    'acq_path_ratio': df['비율'].tolist(),
    'acq_path_detail': df['상세유입경로'].tolist(),
    'acq_path_detail_ratio': df['비율.1'].tolist()
    }

# views 정보를 view_data에서 가져와서 딕셔너리에 추가
for item in view_data:
    article_id = item['article_id']
    views = item['views']
    key = article_id[:12]  # article_id의 앞 12자리를 key로 사용
    
    if key in data_dict:
        # views 값을 숫자로 변환 (가능한 경우에만)
        try:
            views_count = int(views)
        except (ValueError, TypeError):
            views_count = 0  # 변환할 수 없는 경우 views_count를 0으로 설정

        data_dict[key]['views'] = views

        # acq_path 및 acq_path_detail 결합 (해당 필드가 존재하는 경우에만)
        if 'acq_path_ratio' in data_dict[key] and 'acq_path' in data_dict[key]:
            new_acq_path = [f"{path}({ratio}%, {views_count * ratio / 100}건)" for path, ratio in zip(data_dict[key]['acq_path'], data_dict[key]['acq_path_ratio'])]
            data_dict[key]['acq_path'] = new_acq_path

        if 'acq_path_detail_ratio' in data_dict[key] and 'acq_path_detail' in data_dict[key]:
            new_acq_path_detail = [f"{path}({ratio}%, {views_count * ratio / 100}건)" for path, ratio in zip(data_dict[key]['acq_path_detail'], data_dict[key]['acq_path_detail_ratio'])]
            data_dict[key]['acq_path_detail'] = new_acq_path_detail

# 'acq_path_detail_ratio' 및 'acq_path_ratio' 필드를 모든 data_dict 항목에서 제거
for key in data_dict:
    if 'acq_path_detail_ratio' in data_dict[key]:
        del data_dict[key]['acq_path_detail_ratio']
    if 'acq_path_ratio' in data_dict[key]:
        del data_dict[key]['acq_path_ratio']

# Output Preprocessed Data

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data_dict)

# 전치 수행
df_han = df.T

# blog 구분
df_han['category'] = 'blog_hanpro911'
cols = ['category'] + [col for col in df_han.columns if col != 'category']
df_han = df_han[cols]

In [ ]:
df_han['category'] = 'blog_hanpro911'
cols = ['category'] + [col for col in df_han.columns if col != 'category']
df_han = df_han[cols]
df_han

In [ ]:
df_final = pd.concat([df_msp, df_han])

In [ ]:
# Get yesterday's date in the format YYYYMMDD
yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")

# Construct the file name
file_name = rf"C:\Users\ch.kang\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\DailyRawdata\blog_{yesterday}.xlsx"


In [ ]:
# Save the DataFrame to the Excel file with the constructed file name
df_final.to_excel(file_name)